In [1]:
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool

class Agents:

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Finds and summarizes the all markdown files that can solve the problem inside the './Algorithm/Algorithm Content/' path",
            backstory="You are fluent in Korean, and you are very good at finding and summarizing files, summarized information is essential for problem solving",
            verbose=True,
            tools=[
                FileReadTool(),
            ],
        )
    
    def pathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside './Algorithm/' path",
            backstory="You are fluent in Korean, and you are very good at finding files.",
            verbose=True,
            tools=[
                DirectoryReadTool(),
            ],
        )

ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
from crewai import Task

class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the most relevant markdown files for troubleshooting {question}. Provide a summary of markdown files and what is an important point in files.",
            expected_output="Your final answer MUST be a detailed summary of the markdown files in Korean. Include Path to all imported files, why the file was imported, what is an important point and If there is an example for {question}, include it. If you can't find the file, JUST Answer that you don't know",
            agent=agent,
            output_file="algorithmSearch.md",
        )
    
    def pathSearch(self, agent):
        return Task(
            description="Finds the markdown files inside './Algorithm/' path",
            expected_output="Your final answer MUST markdown names",
            agent=agent,
            output_file="algorithMarkdownPath.md",
        )
    

In [ ]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agent = Agents()
tasks = Tasks()

researcher = agent.researcher()
pathSearcher = agent.pathSearcher()

researcher_task = tasks.research(researcher)
pathSearcher_task = tasks.pathSearch(pathSearcher)

crew = Crew(
    agents=[
        pathSearcher,
    ],
    tasks=[
        pathSearcher_task,
    ],
    verbose=2,
)

result = crew.kickoff()